In [1]:
import open3d as o3d 
import numpy as np 
from plyfile import PlyData, PlyElement

In [2]:
# habitat 
import habitat_sim
from dataset.habitat.simulator import init_sim, make_cfg

In [3]:
%%capture  

ply_file = "/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/17DRP5sb8fy/17DRP5sb8fy_semantic.ply"
glb_file = "/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/17DRP5sb8fy/17DRP5sb8fy.glb"

plydata = PlyData.read(ply_file)
scene_o3d = o3d.io.read_point_cloud(ply_file)
sim, _, settings = init_sim(glb_file)

I1120 09:20:25.175065 703534 ManagedFileBasedContainer.h:210] <Dataset>::convertFilenameToPassedExt : Filename : default changed to proposed scene_dataset_config.json filename : default.scene_dataset_config.json
I1120 09:20:25.175084 703534 AttributesManagerBase.h:365] <Dataset>::createFromJsonOrDefaultInternal : Proposing JSON name : default.scene_dataset_config.json from original name : default | This file  does not exist.
I1120 09:20:25.175132 703534 AssetAttributesManager.cpp:120] Asset attributes (capsule3DSolid : capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
I1120 09:20:25.175154 703534 AssetAttributesManager.cpp:120] Asset attributes (capsule3DWireframe : capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
I1120 09:20:25.175175 703534 AssetAttributesManager.cpp:120] Asset attributes (coneSolid : coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_false_useTange

Renderer: GeForce RTX 3080/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 460.91.03
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_ES2_compatibility
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
Using driver workarounds:
    no-forward-compatible-core-context
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits


 dataset  changed to default successfully.
I1120 09:20:25.175663 703534 MetadataMediator.cpp:183] ::setCurrPhysicsAttributesHandle : Old physics manager attributes  changed to ./data/default.physics_config.json successfully.
I1120 09:20:25.175664 703534 MetadataMediator.cpp:68] ::setSimulatorConfiguration : Set new simulator config for scene/stage : /media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/17DRP5sb8fy/17DRP5sb8fy.glb and dataset : default which is currently active dataset.
I1120 09:20:25.224429 703534 ManagedFileBasedContainer.h:210] <Scene Instance>::convertFilenameToPassedExt : Filename : /media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/17DRP5sb8fy/17DRP5sb8fy.glb changed to proposed scene_instance.json filename : /media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/17DRP5sb8fy/17DRP5sb8fy.scene_instance.json
I1120 09:20:25.224566 703534 MetadataMediator.cpp:311] ::getSceneAttributesByName : Dataset : default has no preloaded Sce

In [4]:
vertex = plydata['vertex']
face = plydata['face']

## Generate vertex segmentation 

In [5]:
# collect vertices of one object 
import json 
##################  dictionary format ####################
# def generate_vertex_seg(sim, settings, scene_ply, scene_glb, save_file=None):

#     if sim:
#         settings["scene"] = scene_glb
#         cfg = make_cfg(settings)
#         # load a new scene without closing and opening simulator 
#         sim.reconfigure(cfg)
#     else:
#         sim = init_sim(scene_glb)

#     plydata = PlyData.read(scene_ply)
#     # scene_o3d = o3d.io.read_point_cloud(scene_ply)
#     # NOTE: open3d does not support face color yet, 2021/11/15
#     vertex = plydata['vertex']
#     face = plydata['face']

#     object_ids = [int(obj.id.split("_")[-1]) for obj in sim.semantic_scene.objects]
#     pcl_seg_dict = {obj_id:[] for obj_id in object_ids}

#     for tri in face.data:
#         pcl_seg_dict[tri[1]].append(tri[0]) # append a (3,) numpy int32 array 

#     for obj_id in object_ids:
#         pcl_seg_dict[obj_id] =  np.unique(np.concatenate(pcl_seg_dict[obj_id])).astype(int).tolist()

#     if save_file:
#         with open(save_file, "w") as f:
#             json.dump(pcl_seg_dict, f)
################ list format ######################
def generate_vertex_seg(sim, settings, scene_ply, scene_glb, save_file=None):

    if sim:
        settings["scene"] = scene_glb
        cfg = make_cfg(settings)
        # load a new scene without closing and opening simulator 
        sim.reconfigure(cfg)
    else:
        sim = init_sim(scene_glb)

    plydata = PlyData.read(scene_ply)
    # scene_o3d = o3d.io.read_point_cloud(scene_ply)
    # NOTE: open3d does not support face color yet, 2021/11/15
    vertex = plydata['vertex']
    face = plydata['face']
    num_vertex = len(vertex.data)

    object_ids = [int(obj.id.split("_")[-1]) for obj in sim.semantic_scene.objects]
    pcl_seg_arr = np.zeros(num_vertex, dtype=int) - 1
    pcl_seg_dict = {obj_id:[] for obj_id in object_ids}

    for tri in face.data:
        pcl_seg_dict[tri[1]].append(tri[0]) # append a (3,) numpy int32 array 

    for obj_id in object_ids:
        pcl_seg_dict[obj_id] =  np.unique(np.concatenate(pcl_seg_dict[obj_id])).astype(int)
        # convert to list of labels 
        pcl_seg_arr[pcl_seg_dict[obj_id]] = obj_id

    if save_file:
        # with open(, "w") as f:
        #     json.dump(pcl_seg_dict, f)
        np.savetxt(save_file, pcl_seg_arr, fmt="%d")
    return pcl_seg_arr

In [6]:
# generate_vertex_seg(
#     sim=sim,
#     settings=settings,
#     scene_ply=ply_file,
#     scene_glb=glb_file,
#     save_file="/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/17DRP5sb8fy/17DRP5sb8fy_pclseg.txt"
# )

array([  0,   2,   2, ..., 181, 172, 175])

## visualize segmentation

In [ ]:
# visualize semantic pointclouds with generated annotation
import matplotlib.pyplot as plt

def visualize_pclseg(scene_ply, pclseg_file, color_map="gist_rainbow", save_file=None):
    
    scene_o3d = o3d.io.read_point_cloud(scene_ply)
    pcl_seg = np.loadtxt(pclseg_file)

    # TODO:there are so many objects in one scene, color them by its class label 
    labels = np.unique(pcl_seg)
    max_label = labels.max()
    labels[labels < 0] = max_label + 1 # -1 indicates non-labeled points 

    print(f"scene {scene_ply.split('/')[-2]} has {max_label + 1} objects")
    colors = plt.get_cmap(color_map)(pcl_seg / (max_label+1))

    scene_o3d.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([scene_o3d],
                                    zoom=0.455,
                                    front=[-0.4999, -0.1659, -0.8499],
                                    lookat=[2.1813, 2.0619, 2.0999],
                                    up=[0.1204, -0.9852, 0.1215])
    
    if save_file:
        o3d.io.write_point_cloud(save_file, scene_o3d, write_ascii=True)

## Visualize bbox with pointclouds

In [12]:
from scipy.spatial.transform import Rotation as R
from habitat_sim.utils.common import quat_from_two_vectors, quat_to_coeffs
import quaternion as qt

quat = quat_from_two_vectors(np.array([0, -1, 0]), np.array([0, 0, -1]))
yz_flip_mat = qt.as_rotation_matrix(quat)
zy_flip_mat = qt.as_rotation_matrix(
    quat_from_two_vectors(np.array([0, 0, -1]), np.array([0, -1, 0])))


In [15]:
# mp3d_axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3, origin=[0, 0, 0])
# habitat_axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 5]).rotate(zy_flip_mat)
# o3d.visualization.draw_geometries([mp3d_axis_pcd, habitat_axis_pcd])
# zy_flip_mat

In [24]:
vis_list = []
# visualize coord 
axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 5])
vis_list.append(axis_pcd)

# visualize scene point clouds in habitat coord frame 
scene_o3d_habitat = o3d.geometry.PointCloud(scene_o3d)
scene_o3d_habitat.rotate(zy_flip_mat)
vis_list.append(scene_o3d_habitat)

# # visualize level abb 
# scene_abb = sim.semantic_scene.aabb
# o3d_obb = o3d.geometry.AxisAlignedBoundingBox(scene_abb.center, scene_abb.sizes/2)
# vis_list.append(o3d_obb)

# visualize regions abb
for region in sim.semantic_scene.regions:
    region_aabb = o3d.geometry.AxisAlignedBoundingBox(region.aabb.center, region.aabb.sizes/2)
    region_aabb.color = (1,0,0)
    vis_list.append(region_aabb)
    print(f"only visualize region {region.category.name()}")
    break
# visualize objects obb
# for obj in sim.semantic_scene.objects[:5]:

    # obb = obj.obb
    # center = obb.center
    # extent = obb.sizes
    # r_mat = qt.as_rotation_matrix(qt.as_quat_array(obb.rotation))
    # r_mat = yz_flip_mat @ r_mat
    # o3d_obb = o3d.geometry.OrientedBoundingBox(center, r_mat, extent/2)
    # o3d_obb.color = (0,1,0)
    # vis_list.append(o3d_obb)

# # visualize objects aabb 
# for obj in sim.semantic_scene.objects[:5]:

#     aabb = obj.aabb
#     center = aabb.center
#     sizes = aabb.sizes

#     o3d_aabb = o3d.geometry.AxisAlignedBoundingBox(center, sizes/2)
#     o3d_aabb.rotate(yz_flip_mat)
#     o3d_aabb.color = (0,1,0)
#     vis_list.append(o3d_aabb)  

o3d.visualization.draw_geometries(vis_list)

only visualize region bedroom


In [31]:
# sim.semantic_scene.objects[0].category.name()

'wall'